## 19. 동적 속성과 프로퍼티
### 19.1 동석 속성을 이용한 데이터 랭글링
다음에 나올 몇 가지 예제에서는 OSCON 2014 콘퍼런스에서 오라일리가 공개한 JSON 데이터 피드를 사용하기 위해 동적 속성을 이용한다.

In [ ]:
""" [예제 19-2] osconfeed.py: osconfeed.json 내려받기 """

from urllib.request import urlopen
import warnings
import os 
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg) # 새로 내려받아야 하는 경우 경고 메시지를 출력
        with urlopen(URL) as remote, open(JSON, 'wb') as local: # 두 개의 콘텍스트 관리자를 이용해서 원격 파일을 읽고 저장하는 with 문
            local.write(remote.read())
            
    with open(JSON) as fp:
        return json.load(fp) # JSON 파일을 파싱하고 네이티브 파이썬 객체로 반환하며, 이 피드에는 dict, list, str, int 형의 데이터가 있음

In [ ]:
feed = load()
sorted(feed['Schedule'].keys())

In [ ]:
for key, value in sorted(feed['Schedule'].items()):
    print("{:3} {}".format(len(value), key))

In [ ]:
feed['Schedule']['speakers'][-1]['name'] # 마지막 발표자의 이름

### 19.1.1 동적 속성을 이용해서 JSON과 유사한 데이터 둘러보기
[예제 19-2]는 아주 간단하지만 feed['Schedule']['event'][40]['name']과 같은 구문은 번거롭다. 예제 19-5의 FrozenJSON을 사용하면 자바스크립트처럼 feed.Schedule.event[40].name과 같은 구문으로 동일한 값을 가져올 수 있다. 

FrozenJSON의 핵심은 \_\_getattr\_\_( ) 메서드다.  \_\_getattr\_\_( ) 특별 메서드는 속성을 가져오기 위한 일반적인 과정이 실패할 때(즉, 지명한 속성을 객체, 클래스, 슈퍼클래스에서 찾을 수 없을 때)만 인터프리터에서 호출한다.

In [ ]:
""" [예제 19-5] explore0.py: JSON 데이터셋을 내포한 FrozenJSON 객체, 리스트, 기본형을 담고 있는 FrozenJSON으로 변환 """

from collections import abc

class FrozenJSON:
    """
    점 표기법을 이용해서 JSON과 유사한 객체를 순회하는 읽기 전용 퍼사드 클래스
    """
    
    def __init__(self, mapping):
        self.__data = dict(mapping) # 딕셔너리 메서드를 사용할 수 있으며, 원본을 변경하지 않는다.
        
    def __getattr__(self, name): # name 속성이 없을 때만 __getattr__() 메서드가 호출된다.
        if hasattr(self.__data, name):
            return getattr(self.__data, name) # __data에 들어 있는 객체가 name 속성을 가지고 있으면 그 속성을 반환한다.
        else:
            return FrozenJSON.build(self.__data[name])  # 그렇지 않으면 self.__data에 name을 키로 사용해서 항목을 가져오고 가져온 항목에
                                                        # FrozenJSON.build()를 호출한 결과를 반환한다.
        
    @classmethod # 일반적으로 대안 생성자로 @classmethod를 사용한다.
    def build(cls, obj):
        if isinstance(obj, abc.Mapping): # obj가 매핑형이면 이 객체로부터 FrozenJSON 객체를 생성한다.
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence): # obj가 MutableSequence 형이면 리스트이므로 obj안에 있는 모든 항목에 build() 메서드를 적용해서 생성된 객체들의 리스트를 반환한다.
            return [cls.build(item) for item in obj]
        else:
            return obj # obj가 매핑도 아니고 리스트도 아니면 항목을 그대로 반환한다.

In [ ]:
raw_feed = load()
feed = FrozenJSON(raw_feed)
len(feed.Schedule.speakers)

In [ ]:
sorted(feed.Schedule.keys())

In [ ]:
for key, value in sorted(feed.Schedule.items()):
    print("{:3} {}".format(len(value), key))

In [ ]:
feed.Schedule.speakers[-1].name

In [ ]:
talk = feed.Schedule.events[40]
type(talk) # JSON 객체였지만 이제는 FrozenJSON 객체가 되었다.

In [ ]:
talk.name

In [ ]:
talk.speakers

In [ ]:
talk.flavor # 없는 속성을 읽으려 시도하면 일반적으로 발생하는 AttributeError 대신 KeyError 예외가 발생한다.

### 19.1.2 잘못된 속성명 문제
FrozenJSON 클래스는 파이썬 키워드가 속성명으로 사용된 경우를 처리하지 못한다. 때문에 전달된 매핑 안의 키가 파이썬 키워드인지 검사하고, 파이썬 키워드인 경우에는 뒤에 \_를 붙여 속성을 읽을 수 있게 만드는 것이 좋다.

In [ ]:
""" [예제 19-6] 파이썬 키워드인 속성명에 언더바 붙이기 """

def __init__(self, mapping):
    self.__data = {}
    for key, value in mapping.items():
        if keyword.iskeyword(key):
            key += '_'
        self.__data[key] = value

In [ ]:
x = FrozenJSON({'2be':'or not'})

In [ ]:
x.2be

### 19.1.3 \_\_new\_\_( )를 이용한 융통성 있는 객체 생성
이전에 build() 클래스에서 처리하던 논리를 \_\_new\_\_( ) 메서드로 옮긴 새로운 버전의 FronzenJson 클래스를 보여준다.

In [ ]:
""" [예제 19-7] FronzenJson 객체든 아니든 새로운 객체를 생성하는 대신 __new__() 사용하기 """

from collections import abc
from keyword import iskeyword

class FrozenJSON:
    """
    점 표기법을 이용해서 JSON과 유사한 객체를 둘러보기 위한 읽기 전용 퍼사드 클래스
    """
    
    def __new__(cls, arg): 
        """
        클래스 메서드로서 첫번째 인수는 클래스 자신, 나머지 인수는 __init__() 과 동일하다.
        """
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls) # 기본적으로 슈퍼클래스에 위임한다. 
                                        # 이 경우 FrozenJSON을 인수로 전달하고 object 클래스에서 __new__() 메서드를 호출한다. 
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg
        
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if iskeyword(key):
                key += '_'
            self.__data[key] = value
            
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name]) # 생성자 호출

In [ ]:
raw_feed = load()
feed = FrozenJSON(raw_feed)
len(feed.Schedule.speakers)

In [ ]:
for key, value in sorted(feed.Schedule.items()):
    print("{:3} {}".format(len(value), key))

#### 19.1.4 shelve를 이용해서 OSCON 피드 구조 변경하기
shelve.open() 고위 함수는 shelve.Shelf 객체를 반환한다. shelve.Shelf는 dbm 모듈을 이용해서 키-값 객체를 보관하는 단순한 객체로서 아래과 같은 특징이 있다. 우리는 Json 파일에서 레코드를 모두 읽어 shelve.Shelf에 저장할 것이다. 각 키는 'event.33950' 처럼 레코드 유형과 일련번호로 만들며, 값은 이제 설명할 대로 만든 Record 클래스 객체가 된다.

+ shelve.Shelf는 abc.MutableMapping 클래스를 상속하므로, 매핑형이 제공하는 핵심 메서드들을 제공한다.
+ shelve.Shelf는 sync(), close() 등의 입출력을 관리하는 메서드도 제공한다. 콘텍스트 관리자이기도 하다.
+ 새로운 값이 키에 할당될 때마다 기와 값이 저장된다.
+ 깂은 반드시 pickle 모듈이 처리할 수 있는 객체여야 한다.

In [ ]:
""" [예제 19-9] shedule1.py: shelve.Shelf에 저장된 OSCON 일정 데이터 둘러보기 """
import warnings

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs) # 키워드 인수로부터 생성된 속성으로 객체를 생성할 때 간편히 사용하는 방법
        
def load_db(db):
    raw_data = load()
    warnings.warn("loading" + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1] # 키값의 마지막 s자를 제거한다.(confrerence, event, speaker, venue)
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial']) 
            record['serial'] = key
            # print(record) # for debugging
            db[key] = Record(**record) # Recode 객체를 생성하고, 해당 key로 데이터베이스에 저장한다.

In [ ]:
import shelve

db = shelve.open(DB_NAME)
if CONFERENCE not in db:
    load_db(db)

In [ ]:
speaker = db['speaker.3471']
type(speaker)

In [ ]:
speaker.name, speaker.twitter

In [ ]:
db.close()

Record.\_\_init\_\_() 메서드는 널리 사용되는 파이썬 꼼수를 보여준다. \_\_slots\_\_ 속성이 클래스에 선언되어 있지 않은 한 객체의 \_\_dict\_\_에 속성들이 들어 있다. 따라서 객체의 \_\_dict\_\_를 직접 매핑형으로 설정하면, 그 객체의 속성 묶음을 빠르게 정의할 수 있다.

#### 19.1.5 프로퍼티를 이용해서 연결된 레코드 읽기
shelf에서 가져온 event 레코드의 venue나 speakers 속성을 읽을 때 일련번호 대신 온전한 레코드 객체를 반환하는 것이 이번 버전의 목표이다. 

In [ ]:
""" [예제 19-11] schedule2.py: 임포트, 상수, 개선된 Record 클래스 """
import warnings
import inspect

DB_NAME = 'data/schedule2_db'
CONFERENCE = 'conference.115'

class Record:
    """
    기존 Record 클래스에 __eq__() 함수를 추가했다.
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    
    def __eq__(self, other):
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

In [ ]:
""" [예제 19-12] schedule2.py: MissingDatabaseError와 DbRecord 클래스 """
class MissingDatabaseError(RuntimeError):
    """
    필요한 데이터베이스가 설정되어 있지 않을 때 발생
    """
    
class DbRecord(Record):
    
    __db = None
    
    @staticmethod
    def set_db(db):
        DbRecord.__db = db
        
    @staticmethod
    def get_db():
        return DbRecord.__db
    
    @classmethod
    def fetch(cls, ident): # 클래스 메서드이므로 서브 클래스에서 쉽게 커스터마이즈할 수 있다.
        db = cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg = "database not set; call '{}.setdb(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:
                raise # 나머지 예외는 처리할 수 없으므로 다시 발생 
    
    def __repr__(self):
        if hasattr(self, 'serial'):
            cls_name = self.__class__.__name__
            return "<{} serial={!r}>".format(cls_name, self.serial)
        else:
            return super().__repr__() # serial 속성이 없으면 슈퍼클래스의 __repr__() 메서드를 사용한다. 

In [ ]:
""" [예제 19-13] schedule2.py: Event 클래스 """
class Event(DbRecord):
    
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key)
    
    @property
    def speakers(self):
        if not hasattr(self, '_speaker_objs'):
            spkr_serials = self.__dict__['speakers'] # 속성이 없으면 무한 재귀호출을 방지하기 위해 __dict__객체 'speakers' 속성을 바로 가져온다.
            fetch = self.__class__.fetch # fetch 클래스 메서드에 대한 참조를 가져온다. 키가 fetch인 경우 함수를 읽지 못하는 문제를 방지한다.
            self._speaker_objs = [fetch('speaker.{}'.format(key)) for key in spkr_serials]
        return self._speaker_objs
    
    def __repr__(self):
        if hasattr(self, 'name'):
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__()

In [ ]:
""" [예제 19-14] schedule2.py: load_db() 함수 """
def load_db(db):
    raw_data = osconfeed.load()
    warnings.warn('loading' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        cls_name = record_type.capitalize()
        cls = globals().get(cls_name, DbRecord) # 모듈의 전역 범위에서 그 클래스명의 객체를 가져온다. 그런 이름의 객체가 없다면 DbRecord를 가져온다.
        if inspect.isclass(cls) and issubclass(cls, DbRecord):
            factory = cls
            print(factory)
        else:
            factory = DbRecord
            print(factory)
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            print(factory)
            db[key] = factory(**record)

In [ ]:
DbRecord.set_db(db) # DbRecord는 Record를 상속해서 데이터베이스를 지원한다. 
event = DbRecord.fetch('event.33950') # 어떠한 종류의 레코드든 관계없이 가져온다.
event # DbRecord 클래스를 상속한 Event클래스 객체다.

In [ ]:
event.venue_serial # DbRecord 객체가 반환된다.

In [ ]:
event.venue.name # 자동으로 dereference 하는 것이 이 예제의 목표이다. 

In [ ]:
for spkr in event.speakers: # event.speakers 리스트를 반복해서, 각 발표자를 나타내는 DbRecord 객체들도 가져올 수 있다.
    print('{0.serial}: {0.name}'.format(spkr))

### 19.2 속성을 검증하기 위해 프로퍼티 사용
#### 19.2.1 LineItem 버전 #1
상점의 시스템으로 가정하다. 이 시스템에서 각각의 주문에는 일련의 품목명(line item)이 들어가며, 각 품목명은 [예제 19.15]의 클래스로 표현한다.

In [ ]:
""" [예제 19-15] bulkfood_v1.py: 기본적인 LineItem 클래스 """
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price  = weight
        
    def subtotal(self):
        return self.weight + self.price

In [ ]:
raisins = LineItem('Golden raisins', 10, 6.95)
print(raisins.subtotal())

raisins.weight = -20 # 쓰레기 값이 들어가니
print(raisins.subtotal()) # 쓰레기 값이 나온다.

#### 19.2.2 LineItem 버전 #2: 검증하는 프로퍼티
프로퍼티를 구현하면 게터와 세터 메서드를 사용할 수 있지만 LineItem의 인터페이스는 바뀌지 않는다. [예제 19-17]은 읽기/쓰기 가능한 weight 프로퍼티 코드를 보여준다.

In [ ]:
""" [예제 19-17] bulkfood_v1.py: weight 프로퍼티를 가진 LineItem """
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight # 프로퍼티 세터가 사용되어 음수가 입력되지 못하도록 막는다.
        self.price  = weight
        
    def subtotal(self):
        return self.weight + self.price
    
    @property # 게터 메서드
    def weight(self):
        return self.__weight
    
    @weight.setter  # 세터 메서드
    def weight(self, value):
        if value > 0:
            self.__weight = value # 값이 0보다 크면 비공개 속성인 __weight를 그 값으로 설정한다.
        else:
            raise ValueError('value must be > 0')

In [ ]:
walnuts = LineItem('walnuts', 0, 10.00)

위 예제에는 weight에 예외처리 장치를 마련했지만 price에 적용한다면 반복적인 코드를 입력해야 한다. 14장에서 폴 그레이엄이 "내 프로그램 안에 패턴이 보이면 나는 이것을 문제의 징조라고 생각한다"고 한 말을 상기해보자. 반복을 치료하는 방법은 추상화이며, 프로퍼티 정의를 추상화하려면 프로퍼티 팩토리나 디스크립터를 사용해야한다. 여기서는 프로퍼티 팩토리를 함수로 구현하는 방법을 소개한다.

### 19.3 프로퍼티 제대로 알기
내장된 property()는 비록 데커레이터로 사용되는 경우가 많지만 사실상 클래스다. property() 생성자의 전체 시그너처는 다음과 같다. 모든 인수는 선택적이며, 인수에 함수를 제공하지 않으면 생성된 프로퍼티 객체가 해당 연산을 지원하지 않는다.
```
property(fget=None, fset=None, fdel=None, doc-None)
```

In [ ]:
""" [예제 19-18] bulkfood_v2.py: 데커레이터를 사용하지 않는 LineItem """
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight # 프로퍼티 세터가 사용되어 음수가 입력되지 못하도록 막는다.
        self.price  = weight
        
    def subtotal(self):
        return self.weight + self.price
    
    def get_weight(self):
        return self.__weight
    
    def set_weight(self, value):
        if value > 0:
            self.__weight = value # 값이 0보다 크면 비공개 속성인 __weight를 그 값으로 설정한다.
        else:
            raise ValueError('value must be > 0')
            
    weight = property(get_weight, set_weight) # property 객체를 생성하고 클래스 공개 속성에 할당한다.

### 19.3.1 객체 속성을 가리는 프로퍼티

객체와 클래스가 모두 동일한 이름의 속성을 가지고 있으면, 객체를 통해 속성에 접근할 때 객체 속성이 클래스 속성을 가린다. [예제 19-9]는 이 현상을 잘 보여준다.

In [ ]:
""" [예제 19-19] 객체 속성이 클래스 데이터 속성을 가린다. """

class Class:
    data = 'the clas data attr'
    @property
    def prop(self):
        return 'the prop value'      

In [ ]:
obj = Class()

In [ ]:
vars(obj)

In [ ]:
obj.data

In [ ]:
obj.data = 'bar' # 객체 속성이 클래스 속성을 덮는다.

In [ ]:
vars(obj)

In [ ]:
Class.data # 클래스 속성은 그대로 있다.

In [ ]:
""" [예제 19-20] 객체 속성은 클래스 프로퍼티를 가리지 않는다. """
Class.prop # Class에서 직접 prop을 읽으면 게터 메서드를 통하지 않고 프로퍼티 객체 자체를 가져온다.

In [ ]:
obj.prop # obj.prop을 읽으면 프로터피 게터를 실행한다. 

In [ ]:
obj.prop = 'foo' # 값을 할당할 수는 없다.

In [ ]:
obj.__dict__['prop'] = 'foo' # 직접 'prop'을 설정하면 제대호 작동한다.

In [ ]:
vars(obj) # 두 개의 속성이 있는 것을 알 수 있다.

In [ ]:
obj.prop # 그러나 obj.prop을 읽으면 여전히 프로퍼티 게터가 실행된다. 프로퍼티는 객체 속성에 의해 가려지지 않는다.

In [ ]:
Class.prop = 'baz' # Class.prop을 덮으면 프로퍼티 객체가 제거된다.

In [ ]:
obj.prop # 이제는 객체 속성을 가져온다. Class.prop는 더 이상 프로퍼티가 아니므로 obj.prop을 가리지 않는다.

In [ ]:
""" [예제 19-21] 새로운 클래스 프로퍼티는 기존 객체 속성을 가린다. """
obj.data

In [ ]:
Class.data

In [ ]:
Class.data = property(lambda self: 'the "data" prop value')
obj.data # 프로퍼티가 객체 속성을 가린다.

In [ ]:
del Class.data # 프로퍼티를 제거하면
obj.data       # 다시 객체의 data 속성을 가져온다.

위의 예제를 보면 속성을 검색할 때 obj.\_\_class\_\_에서 시작하고, 클래스 안에 attr이라는 이름의 프로퍼티가 없을 때만 파이썬이 obj 객체를 살펴본다. 이 규칙은 프로퍼티 뿐 아니라 모든 종류의 디스크립터(오버라이딩 디스크립터 포함)에 적용된다는 것을 알아두자.

### 19.3.2 프로퍼티 문서화
콘솔의 help() 함수나 IDE 같은 도구가 프로퍼티에 대한 문서를 보여주어야 할 때 프로퍼티의 \_\_doc\_\_ 속성에서 정보를 가져온다.
```
weight = property(get_weight, set_weight, doc='weight in kilograms')
```
프로퍼티를 데커레이터로 사용하는 경우에는 @property 데커레이터로 장식된 게터 메서드의 문서화 문자열이 프로퍼티 전체의 문서로 사용된다.

In [ ]:
""" [예제 19-22] 프로퍼티에 대한 문서화 """

class Foo:
    
    @property
    def bar(self):
        '''The bar attribute'''
        return self.__dict__['bar']
    
    @bar.setter
    def bar(self, value):
        self.__dict__['bar'] = value
    

In [ ]:
help(Foo.bar)

In [ ]:
help(Foo)

### 19.4 프로퍼티 팩토리 구현하기

0보다 큰 값만 갖도록 quantity()라는 프로퍼티 팩토리를 만든다.

In [4]:
""" [예제 19-23] bulkfood_v2prop.py : quantity() 프로퍼티 팩토리 사용하기 """
def quantity(storage_name): # storage_name 인수는 각 프로퍼티를 어디에 저장할지 결정한다.
    
    def qty_getter(instance):
        return instance.__dict__[storage_name] # 이 메서드가 클래스 본체에 있는 것은 아니므로 저장할 객체 가리킨다. 
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value # gty_getter()가 storage_name을 참조하므로, storage_name은 함수 클로져에 보관된다.
                                                    # 프로퍼티를 사용하면 무한재귀 호출이 되므로 instance.__dict__에서 직접 가져온다.
        else:
            raise ValueError("value must be > 0")
            
    return property(qty_getter, qty_setter)


class LineItem:
    weight = quantity('weight')
    price = quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight + self.price

In [5]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
nutmeg.weight, nutmeg.price

(8, 13.95)

In [6]:
sorted(vars(nutmeg).items())

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

weight 프로퍼티는 weight 객체 속성을 가리므로 self.weight이나 nutmeg.weight로 참조하는 것은 모두 프로퍼티 함수에 의해 처리되며, 객체의 \_\_dict\_\_( )에 접근하는 방법을 이용해야 프로퍼티 논리를 우회할 수 있다.